In [1]:
import sys
import os
import time
import re
import json
import shutil
from SciServer import CasJobs, SkyQuery, SciDrive, SkyServer
from astropy.table import Table,vstack,Column
from astroquery.mast import Catalogs
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.utils.console import ProgressBar
from collections import deque
import numpy as np
from pydoc import locate
from urllib.parse import quote as urlencode
from urllib.request import urlretrieve
import http.client as httplib
import pprint
pp = pprint.PrettyPrinter(indent=4)
import gzip
import tarfile
from IPython.core.display import display
import matplotlib.pyplot as mplplot
import astropy as ap
import astropy.io.fits as astrofits
import astropy.io.votable as astrovot
import astropy.wcs as astrowcs
import astropy.visualization as astrovis
import astropy.visualization.mpl_normalize as astromplnorm
from astropy.nddata.utils import Cutout2D
import time
import math
import pdb
import pandas as pd

Definitions. These are all referenced and explained in the code itself.

In [2]:
def mastQuery(request):
    """Perform a MAST query.
    
        Parameters
        ----------
        request (dictionary): The MAST request json object
        
        Returns head,content where head is the response HTTP headers, and content is the returned data"""
    
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    # Making the query
    conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode('utf-8')

    # Close the https connection
    conn.close()

    return head,content

def getHSCMatches(matchId):

    request = {'service':'Mast.HscMatches.Db.v2',
               'params':{'input':matchId,
                         'cache-breaker':10},
               'format':'json',
               'page':1,
               'pagesize':4}   

    headers,outString = mastQuery(request)

    outData = json.loads(outString)

    return outData

def query_cas():
    query = "SELECT dr8objid, ra, dec FROM zoo2MainSpecz"
    result = CasJobs.executeQuery(query, 'DR12', format='pandas')

    result = Table.from_pandas(result)
    result.write('casquery.ecsv')
    return result

def read_cas(filename='casquery.ecsv'):
    return Table.read(filename)

def query_mast(coords,radius=0.001*u.deg):
    results = Catalogs.query_region(coords,radius=radius)
    print(results)
    
def stringtype(typestr):
    typemap = {'string':str,'float':float,'int':int,'boolean':bool,'date':str}
    return typemap[typestr]
    
def typecast(val,typestr):
    if val is None:
        return None
    elif typestr == 'string':
        return str(val)
    else:
        return locate(typestr)(val)

def mastJson2Table(line):
    jsonObj = line
    if not jsonObj['data']:
        return None
    coldict = {field['name']:stringtype(field['type']) for field in jsonObj['fields']}
    keys,dtypes = zip(*coldict.items())
    
    rows = deque()
    
    for d in jsonObj['data']:
        row = [d[key] if d[key] is not None else '99' for key in keys]
        rows.append(row)
     
    table = Table(rows=list(rows),names=keys,dtype=dtypes)
    return table

def genBundleRequest(imageNames, outfileNamePrefix='downloadBundle', extension='tar.gz'):
    baseUrl = "http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset="
    urlList = ",".join(['{}{}'.format(baseUrl, imageName) for imageName in imageNames])
    pathList = ['{}.fits'.format(imageName) for imageName in imageNames]

    request = {"service":"Mast.Bundle.Request",
               "params":{"urlList":urlList,
                         "filename":outfileNamePrefix,
                         "pathList":pathList,
                         "extension":extension,
                         'cache-breaker':10}
              }
    return request,outfileNamePrefix,extension

def downloadRequest(url):
    server='mast.stsci.edu'
    
    conn = httplib.HTTPSConnection(server)
    conn.request("GET", url)
    resp = conn.getresponse()
    print(resp.getheader)
   
    fileName = resp.getheader('Content-Disposition')[21:]
    fileContent = resp.read()

    with open(fileName,'wb') as FLE:
        FLE.write(fileContent)

    conn.close()

    return fileName

This is used to read in the astropy table that has the ra, dec, match id and other infromation stored.

In [3]:
os.chdir('/home/idies/workspace/persistent')
t = Table.read('final_table.fits')

This section searches through the table for all matches where a spacific filter is used. This filter is selected and a list is made of each match and each image associated with each match for that filter.

In [4]:
filt = 'A_F475W'
matchIDs = [i['MatchID'] for i in t if isinstance(i[filt], np.float64) and i[filt] != 99.0 and not math.isnan(i[filt])]
images = [mastJson2Table(getHSCMatches(str(i)))['ImageName'] for i in matchIDs]

The loops through each image and matches any number of match ids associated with that imaged to one section of a dictionary. This groups multiple matches together with the image.

In [5]:
image_dict = {}
for i,name in enumerate(images):
    image_list = [j for j in name if 'f475w' in j]
    for k in image_list:
        image_dict[k] = [matchIDs[ind] for ind,out in enumerate(images) for In in out if In == k]

This section of code is the actual downloade section. It takes a few of the images in the above dictionary and downloads them to a bundle in a tar.gz format. This file is then extracted and the image is saved to another folder for the time being.

In [27]:
os.chdir('/home/idies/workspace/persistent/Images')
name_list = []
for number,key in enumerate(image_dict):
    name_list.append(key)
query,filename,extension = genBundleRequest(name_list[:4])
pp.pprint(query)
headers,bundleString = mastQuery(query)
bundleInfo = json.loads(bundleString)
pp.pprint(bundleInfo)
downloadfile = downloadRequest(bundleInfo['url'])
tar = tarfile.open("downloadBundle.tar.gz")
os.chdir('/home/idies/workspace/persistent/Images/Images')
tar.extractall()
tar.close()

{   'params': {   'cache-breaker': 10,
                  'extension': 'tar.gz',
                  'filename': 'downloadBundle',
                  'pathList': [   'hst_9401_85_acs_wfc_f475w.fits',
                                  'hst_9401_81_acs_wfc_f475w.fits',
                                  'hst_9401_68_acs_wfc_f475w.fits',
                                  'hst_9401_11_acs_wfc_f475w.fits'],
                  'urlList': 'http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_9401_85_acs_wfc_f475w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_9401_81_acs_wfc_f475w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_9401_68_acs_wfc_f475w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_9401_11_acs_wfc_f475w'},
    'service': 'Mast.Bundle.Request'}
{   'bytesStreamed': 1850309004,
    'fileStatusList': {   'http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_9401_11_acs_wfc_f475w': '{"status":"COMPLETE"}',
           

Due to the unfortinate renaming of a file to "System.String[]" one image in each set must be deleted for use in the future. This could be worked around but the image name is lost and I haven't saved it to a variabel above yet so it breaks the code below... this will be fixed soon.

In [28]:
os.remove('System.String[]')
shutil.rmtree('downloadBundle')

This code loops through the folder containing images and makes cutouts on each match from each image. These cutouts are named after the image file and the number of cutout it is and saved to another folder.

In [38]:
os.chdir('/home/idies/workspace/persistent/Images/Images')
for filename in os.listdir('/home/idies/workspace/persistent/Images/Images'):
    if filename == '.ipynb_checkpoints':
        continue
    print(filename)
    hdu = astrofits.open(filename)
    w = astrowcs.WCS(hdu[1].header)
    print(w)
    name = filename.replace('.fits', '')
    for number,z in enumerate(image_dict[name]):
        for i,j in enumerate(t['MatchID']):
            if j == z:
                ra_cut = t['ra'][i]
                dec_cut = t['dec'][i]
                coord = SkyCoord(ra_cut,dec_cut,unit=(u.deg,u.deg),frame='icrs')
                print('{:.10f}, {:.10f}'.format(ra_cut,dec_cut))
                cut = Cutout2D(hdu[1].data,position=coord,size=4*u.arcsec,wcs=w)
                cut_fits = astrofits.PrimaryHDU(data=cut.data, header=cut.wcs.to_header())
                cut_fits.writeto('/home/idies/workspace/persistent/Images/Cuts/'+name+'_'+str(number)+'_cut.fits')

.nfs0000000e00a08d3300000008
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 10.80791915671064  41.18530442173756  
CRPIX : 2749.999999999824  2744.999999999824  
CD1_1 CD1_2  : -1.3888888888888e-05  -4.8712485898075e-35  
CD2_1 CD2_2  : 0.0  1.38888888888896e-05  
NAXIS : 5499  5489


KeyError: '.nfs0000000e00a08d3300000008'